<a href="https://colab.research.google.com/github/sanhapon/llm_20_questions/blob/main/llm_20_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")

In [3]:
import torch

if torch.cuda.is_available():
  model = model.to("cuda")

In [42]:
chat = [
      { "role": "user", "content":
          f"""We are playing the 20 Questions game. The questioner's goal is to ask the question and guess for secret. The secret in the game is the country or city name.
          The sample question: Is it a Asia continent?
          Now, you are the questioner, generate a yes-or-no question to find the place. Enclose your question in **.
          """
      },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
output_sequences = model.generate(
    input_ids=inputs.to(model.device),
    max_new_tokens=150,
    do_sample=True,
    temperature=0.1,
    top_k=1,
    top_p=0.1,
)

outputs = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(outputs)

user
We are playing the 20 Questions game. The questioner's goal is to ask the question and guess for secret. The secret in the game is the country or city name.
          The sample question: Is it a Asia continent? 
          Now, you are the questioner, generate a yes-or-no question to find the place. Enclose your question in **.
Is it a city in the state of California, USA?


In [49]:
chat = [
    { "role": "user", "content":
       """Convert the yes-or-no question to declarative sentence, for example: "Is it a cat?", "Yes". The declarative sentence would be "It is a cat".
        Here is the yes-or-no question from previous round: "Is it a city in the state of California, USA?",  "Yes".
        Enclose your declarative sentence with *..
        """
      },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
output_sequences = model.generate(
    input_ids=inputs.to(model.device),
    max_new_tokens=50,
    do_sample=True,
    temperature=0.1,
    top_k=1,
    top_p=0.1
)

outputs = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(outputs)

user
Convert the yes-or-no question to declarative sentence, for example: "Is it a cat?", "Yes". The declarative sentence would be "It is a cat".
        Here is the yes-or-no question from previous round: "Is it a city in the state of California, USA?",  "Yes".
        Enclose your declarative sentence with *..
Sure, here is the declarative sentence for the yes-or-no question "Is it a city in the state of California, USA?"

*It is a city in the state of California, USA.*


In [50]:
chat = [
    { "role": "user", "content":
     f"""We are continuing playing the 20 Questions game.
        The sample question: Is it a Asia continent?
        Based on this information \"It is a city in the state of California, USA.\"
        Now, you are the questioner, generate yes-or-no question to find "specific" place. Enclose your question with *.
        """
      },

    # The secret relates to {"It is a country in the Middle East."}
    # Now, you are the questioner, generate yes-or-no question to narrow scope of the secret place.
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
output_sequences = model.generate(
    input_ids=inputs.to(model.device),
    max_new_tokens=100,
    do_sample=True,
    temperature=0.1,
    top_k=1,
    top_p=0.1
)

outputs = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(outputs)


user
We are continuing playing the 20 Questions game.
        The sample question: Is it a Asia continent? 
        Based on this information "It is a city in the state of California, USA."
        Now, you are the questioner, generate yes-or-no question to find "specific" place. Enclose your question with *.
Is it a city in the state of California, USA that is known for its beautiful beaches?


In [98]:
chat = [
    { "role": "user", "content":
     f"""We are continuing playing the 20 Questions game.
        Based on this information \"It is the beach in the Southern of Thailand\".
        Guess the place based on above information.
        """
      },
]

    # Based on this information \"It is a country in the Middle East that is known for its ancient ruins and is also a popular tourist destination in the **desert**\"

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
output_sequences = model.generate(
    input_ids=inputs.to(model.device),
    max_new_tokens=100,
    do_sample=True,
    temperature=0.1,
    top_k=1,
    top_p=0.1
)

outputs = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(outputs)


user
We are continuing playing the 20 Questions game.
        Based on this information "It is the beach in the Southern of Thailand".
        Guess the place based on above information.
Songkla Beach is located in the Southern province of Thailand. It is known for its pristine beaches, turquoise waters, and vibrant nightlife.
